# Possible solution to exercise 6.1

## Setup

In [87]:
import pandas as pd
import altair as alt

## Data import

In [88]:
df = pd.read_csv("data.csv")
df

,Date,Company,nps
0,2019-01-01,OUR COMPANY,29.0
1,2019-02-01,OUR COMPANY,29.0
2,2019-03-01,OUR COMPANY,31.0
3,2019-04-01,OUR COMPANY,34.0
4,2019-05-01,OUR COMPANY,34.0
...,...,...,...
65,2019-10-01,COMPETITOR D,28.0
66,2019-11-01,COMPETITOR D,28.0
67,2019-12-01,COMPETITOR D,29.0
68,2020-01-01,COMPETITOR D,32.0


## Data preparation

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     70 non-null     object 
 1   Company  70 non-null     object 
 2   nps      70 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.8+ KB


In [90]:
# Change data format
df['Company'] = df['Company'].astype('category')
df['Date'] =  pd.to_datetime(df['Date'])

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     70 non-null     datetime64[ns]
 1   Company  70 non-null     category      
 2   nps      70 non-null     float64       
dtypes: category(1), datetime64[ns](1), float64(1)
memory usage: 1.5 KB


## Chart

### 1) Prepare line chart

In [91]:
# Get names of companies
COMPANY = df['Company'].cat.categories.to_list()
COMPANY

['COMPETITOR A', 'COMPETITOR B', 'COMPETITOR C', 'COMPETITOR D', 'OUR COMPANY']

In [92]:
# Prepare colors
colors = alt.Scale(
  domain= COMPANY,
  range=['grey', 'grey', 'grey', 'grey', 'black']
)
colors


Scale({
  domain: ['COMPETITOR A', 'COMPETITOR B', 'COMPETITOR C', 'COMPETITOR D', 'OUR COMPANY'],
  range: ['grey', 'grey', 'grey', 'grey', 'black']
})

In [93]:
# Create line chart
line_chart = alt.Chart(df).mark_line().encode(
    x=alt.X("Date:T",
            axis=alt.Axis(title='Time', 
                          titleAnchor="start",
                          labelAngle=-90,
                          grid=False,
                          format="%b %y"),
                          ),
    y=alt.Y("nps:Q",
            scale=alt.Scale(domain=[0, 60]),
            axis=alt.Axis(title = "NPS",
                          grid=False,
                          titleAnchor="end",
                          values=[0,10,20,30,40,50,60])
                        ),
    color=alt.Color("Company:N", scale=colors, legend=None),
    strokeWidth=alt.value(4)
).properties(
     title={
      "text": ['NPS: we rank 4th among competitors'], 
      "subtitle": ["Net Promoter Score (NPS) over time"],
      "color": "black",
      "subtitleColor": "grey"
    },
    width=550,
    height=350
)

line_chart


alt.Chart(...)

### 2) Create points at the end of the line

In [94]:
# Create a new dataframe wich only includes the last row of the data
df_2 = df[df["Date"]== "2020-02-01"]

points = alt.Chart(df_2).mark_circle(opacity=1, size=150).encode(
    alt.X("Date:T"),
    alt.Y("nps:Q"),
    color=alt.Color("Company:N", scale=colors)
)

line_chart + points

alt.LayerChart(...)

### 3) Create labels

In [95]:
# Create base chart for our labels
label_base =  alt.Chart(df_2).mark_text().encode(
    alt.X("Date:T", aggregate="max"),
    alt.Y("nps:Q", aggregate={"argmax": "Date"}),
    alt.Color("Company:N", legend=None) 
)

# Create labels for nps values
label_nps = label_base.mark_text(align="left", 
                           dx=10, 
                           size=14, 
                           fontWeight="bold"
            ).encode(alt.Text("nps:Q")) 

# Create labels for companies
label_company = label_base.mark_text(align="left", 
                               dx=30, 
                               size=14, 
                               fontWeight="bold"
            ).encode(alt.Text("Company:N"))

label_base + label_nps + label_company

alt.LayerChart(...)

### 4) Combine all elements

In [96]:
alt.layer(line_chart, points, label_nps, label_company).configure_view(
    strokeWidth=0
).configure_title( 
    fontSize=22,
    font='Arial',
    anchor='start',
    fontWeight="normal"
).configure_axis(
    labelFont='Arial',
    titleFont='Arial',
    labelFontSize=11,
    titleFontSize=16,
    titleFontWeight="normal",
    titleColor="grey"
)


alt.LayerChart(...)